# Find hospitals closest to an incident

This jupyter notebook constitutes the Exercise Sheet for the first lecture aput the arcgis API for Python.

It is build as a 'gap text' challenge, there is a lot of text, code snippets and comments already added. To complete this exercise follow the instructions to build a fully working jupyter notebook. Add comments if necessary. 

You will use the functions of the network analyst module from the arcgis api for Python to build a solver the finds the route and the distance in kilometers towards the next hospital in Münster. 

There are two bonus challenges for 2 points each:
1. Build a Interactive Input for the Incident Coordinates
2. Label the result of the routing with the kilometer distance

You will find the details for the bonus challenge in the explanations.

There are 10 Points for the exercise and 4 bonus points available.
Please remember to write code that is easy readable, well documented and actually working. 

The `network` module of the ArcGIS API for Python can be used to solve different types of network analysis operations. In this Exercise, we see how to find the hospital that is closest to an incident.

## Closest facility

The closest facility solver provides functionality for finding out the closest locations to a particular input point. This solver would be useful in cases when you have an incident and need to find the closest facility or need to get information on the travel time and the distance to each of the facilities from an incident point for reporting purposes.

![](http://desktop.arcgis.com/en/arcmap/latest/extensions/network-analyst/GUID-96C273DB-6A24-4D42-AADA-975A33B44F3D-web.png)

When finding closest facilities, you can specify how many to find and whether the direction of travel is toward or away from them. The closest facility solver displays the best routes between incidents and facilities, reports their travel costs, and returns driving directions.

### Connect to your GIS
As a first step, you would need to establish a connection to your organization which could be an ArcGIS Online organization or an ArcGIS Enterprise.

In [1]:
from IPython.display import HTML
import pandas as pd
from arcgis.gis import GIS
# import arcpy
# arcpy.CheckOutExtension("Network")

#connect to your GIS 
# please dont enter your password in clear letters, make sure you use getpass
import getpass

# connecting with my GIS
password = getpass.getpass("Password please:")
my_gis = GIS('https://www.arcgis.com', 'o.bin.zamir_UNI-MS', password)

### Create a Network Layer
To perform any network analysis (such as finding the closest facility, the best route between multiple stops, or service area around a facility), you would need to create a `NetworkLayer` object. In this sample, since we are solving for closest facilities, we need to create a `ClosestFacilityLayer` which is a type of `NetworkLayer`.

To create any `NetworkLayer` object, you would need to provide the URL to the appropriate network analysis service. Hence, in this sample, we provide a `ClosestFacility` URL to create a `ClosestFacilityLayer` object. 

Since all ArcGIS Online organizations already have access to those routing services, you can access this URL through the `GIS` object's `helperServices` property. If you have your own ArcGIS Server based map service with network analysis capability enabled, you would need to provide the URL for this service.

Let us start by importing the `network` module

In [2]:
import arcgis.network as network

Access the analysis URL from the `GIS` object

In [3]:
analysis_url = my_gis.properties.helperServices.closestFacility.url
analysis_url

'https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World'

Create a `ClosestFacilityLayer` object using this URL

See the help for details in Syntax. https://developers.arcgis.com/python/api-reference/arcgis.network.toc.html#closestfacilitylayer

In [4]:
# creating the closest facilitylayer object
cf_layer = network.ClosestFacilityLayer(analysis_url, gis= my_gis)

In [5]:
# lets see the cf_layer
cf_layer

<ClosestFacilityLayer url:"https://route.arcgis.com/arcgis/rest/services/World/ClosestFacility/NAServer/ClosestFacility_World">

### Create hospitals layer
In this sample, we will be looking for the closest hospital (facility) to an incident location. Even though we are interested in finding out the closest one, it would still be helpful to get the information on the distance and travel time to all of them for reference purposes.

In the code below, we need to geocode the hospitals' addresses as well as do the reverse geocode for the incident location which has been supplied in the latitude/longitude format.

To perform the geocode operations, we import the `geocoding` module of the ArcGIS API.

In [6]:
from arcgis import geocoding

In this sample, we geocode addresses of hospitals to create the facility layer. In your workflows, this could any feature layer. Create a list of hospitals in Münster, Germany

In [7]:
hospitals_addresses = ['Dorbaumstraße 300, 48157 Münster, Germany',
                       'Alexianerweg 9, 48163 Münster, Germany',
                       'Westfalenstr. 109, 48165 Münster, Germany',
                       'Loerstraße 23, 48143 Münster, Germany',
                       'Hohenzollernring 70, 48145 Münster, Germany',
                       'Wichernstraße 8, 48147 Münster, Germany',
                       'Domagkstr. 5, 48149 Münster, Germany']

Loop through each address and geocode it. The geocode operation returns a list of matches for each address. We pick the first result and extract the coordinates from it and construct a `Feature` object out of it. Then we combine all the `Feature`s representing the hospitals into a `FeatureSet` object.

Geocoding syntax: https://developers.arcgis.com/python/api-reference/arcgis.geocoding.html#geocode

Feature Syntax: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#feature

Feature Set Syntax: https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#featureset

Hint: Check the structure from the return element of the geocoder to properly construct your feature. Use a loop to run through the list of adresses, add each result feature to a python list and use this python list to build your feature set. While building the feature set, use this as the value for the spatial_reference parameter `{'wkid' : 4326, 'latestWkid': 4326}`

In [8]:
from arcgis.features import Feature, FeatureSet

In [9]:
# let first make a list to store the hospital feature individually
h_feat_list = []

# lets loop through the addresses and geocode them:
for h_address in hospitals_addresses:
    geocoded = geocoding.geocode(h_address)[0]
    # for each h_address lets make a hospital feature
    h_feature = Feature(
        geometry = geocoded['location'], # lets store the values 
        attributes = {'name': h_address} # storing the address as attribute
    )
    # now at last lets append this feature to the list
    h_feat_list.append(h_feature)
    


In [10]:
# let see first our list
for feat in h_feat_list:
    print(feat)

{"geometry": {"x": 7.709353094785, "y": 52.013365641243}, "attributes": {"name": "Dorbaumstra\u00dfe 300, 48157 M\u00fcnster, Germany"}}
{"geometry": {"x": 7.564906919172, "y": 51.877521980497}, "attributes": {"name": "Alexianerweg 9, 48163 M\u00fcnster, Germany"}}
{"geometry": {"x": 7.634180079975, "y": 51.908846032763}, "attributes": {"name": "Westfalenstr. 109, 48165 M\u00fcnster, Germany"}}
{"geometry": {"x": 7.629749070681, "y": 51.959485007431}, "attributes": {"name": "Loerstra\u00dfe 23, 48143 M\u00fcnster, Germany"}}
{"geometry": {"x": 7.64876467532, "y": 51.960599004273}, "attributes": {"name": "Hohenzollernring 70, 48145 M\u00fcnster, Germany"}}
{"geometry": {"x": 7.624749097798, "y": 51.972945421922}, "attributes": {"name": "Wichernstra\u00dfe 8, 48147 M\u00fcnster, Germany"}}
{"geometry": {"x": 7.600980451145, "y": 51.96145798171}, "attributes": {"name": "Domagkstr. 5, 48149 M\u00fcnster, Germany"}}


Construct a `FeatureSet` using each hospital `Feature`.

In [11]:
# now we can mkae a feature set from it
hospitals_fset = FeatureSet(
    features = h_feat_list, #now we will give our list here
    # now lets set coordinate reference system also as given 
    spatial_reference={
        'wkid': 4326,
        'latestWkid': 4326
    }
)

Lets draw our hospitals on a map. To do so, intanciate a gis.map object, focused in Münster. Set the basemap to `arcgis-light-gray`

Details about the map widget can be found here https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.GIS.map
use the map.draw() function to add the hospitals to the map. You can use the following value for the 'symbol' parameter: `{"type": "esriSMS","style": "esriSMSSquare","color": [76,115,0,255],"size": 8,}`

In [12]:
from IPython.display import HTML

In [71]:
# instanciate the map
map1 = my_gis.map('Münster') # this is our map
map1.basemap = 'arcgis-light-gray' # setting basemap style as given
map1

MapView(layout=Layout(height='400px', width='100%'))

In [72]:
# lets first set the parameters for the symbol as given instructions
syb = {
    "type": "esriSMS",
    "style": "esriSMSSquare",
    "color": [76,115, 0,255],
    "size": 8
}
# draw the hospitals on the map
map1.draw(hospitals_fset, symbol=syb)

### Create incidents layer
Lets use the geocoding.reverse_geocoding function to build an Feature set from the provided incident coordinates. Feel free to use your own coordinates or use the one provided

The syntax for the reverse geocoding is here: https://developers.arcgis.com/python/api-reference/arcgis.geocoding.html#reverse-geocode

1. Reverse Geocode the coordinates
2. build a arcgis.Feature from the result object, using the 'location' from the response object as a parameter for the `geometry` parameter and the 'adress' from the response object for the `attributes` parameter of the Feature.
3. Build a Feature set from the one Feature, similar to the feature set for the hospitals. You can use the same spatial_reference parameter

In [15]:
# its time for incident layer
incident_layer = Feature()

## Bonus Challenge: Build an interactive Input for the coordinates
for 2 extra points: 
Open an input form and verify that the input string has the format of XX.XXXX, YY.YYYY

Give Feedback to the user if the format is correct or not.

In [16]:
# reverse geocode & build feature
incident_coords = '7.641378810646356,51.95304243253929'

# let take the coordinates individually
# print(int(len(incident_coords)-1)/2)
# x = incident_coords[(len(incident_coords)-1/2):]
# print(x)

# work on this later
long = "7.641378810646356"
lat = "51.95304243253929"


# lets reverse geocode to get the address from this coordinates:
rev_geocode = geocoding.reverse_geocode({"x": long, "y": lat})

In [17]:
incident_layer.geometry = rev_geocode['location'] # reteriving the location
incident_layer.attributes = {'address': rev_geocode['address']['Match_addr']} # for single exact location

In [18]:
# build the incident feature set from the layer
incident_f_set = FeatureSet(
    features=[incident_layer],
    spatial_reference={
        'wkid': 4326,
        'latestWkid': 4326
    } # same as hospital feature set
)

In [19]:
# check for the result (this cell does not need to be changed)
incident_f_set.features

[{"geometry": {"x": 7.641618030163, "y": 51.953028183304, "spatialReference": {"wkid": 4326, "latestWkid": 4326}}, "attributes": {"address": "Dortmunder Stra\u00dfe 34, 48155, M\u00fcnster, Mitte-Nordost, Mauritz, Nordrhein-Westfalen", "OBJECTID": 1}}]

Draw the incident Feature Set on the map, using this value for the 'symbol' parameter `{"type": "esriSMS","style": "esriSMSCircle","color": [255,0,0,255],"size": 8}`

In [73]:
# draw the incidents on the existing map widget
i_syb = {
    "type": "esriSMS",
    "style": "esriSMSCircle",
    "color": [255,0,0,255],
    "size": 8
}
# now let add it to the map
map1.draw(incident_f_set, symbol= i_syb)

## Solve for closest hospital
Now, you can solve the closest facility layer you created in the beginning. Syntax: https://developers.arcgis.com/python/api-reference/arcgis.network.toc.html#closestfacilitylayer

Use your incidents feature set for the `incidents` parameter, your hospital feature set for the `facilities` parameter.

Set the `default_target_facility_count` parameter to 1, since we only want to see the routing to the closest hospital

Set the remaining attributes in a way that the impedance is set to travel time and the travel distance is added in kilometers


In [23]:

# solve your closest facillity layer
result = cf_layer.solve_closest_facility(
    incidents= incident_f_set,
    facilities=hospitals_fset,
    default_target_facility_count= 1, # we need only thhe closest hospital
    return_cf_routes=True,
    directions_length_units = 'kilometers',
    impedance_attribute_name = 'TravelTime'
    
)

Let us inspect the result dictionary. You will use the content of the `routes` dictionary to create the line to be plotted on the map. you can keep the following 3 Code Cells as is, without doing anything

In [24]:
result.keys()

dict_keys(['requestID', 'routes', 'messages'])

Let us use the `routes` dictionary to construct line features out of the routes to display on the map

In [25]:
result['routes'].keys()

dict_keys(['fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features'])

In [26]:
result['routes']['features'][0].keys()

dict_keys(['attributes', 'geometry'])

From here, you need to continue to modify the code

Construct line features out of the routes that are returned. 

The process is the same as before. 

Build a `Feature` for each route (there yould only be one)
Add this to a result list
Build a `FeatureSet` from the list

In [28]:
# build the list with features
line_feat_list = []
for line_dict in result['routes']['features']:
    line_feat = Feature(geometry=line_dict['geometry'])
    line_feat_list.append(line_feat)
                       


In [29]:
# create a feature set
# lets repeat the steps as we did before
routes_fset = FeatureSet(
    line_feat_list, # we will give the line list with the same spatial reference
    spatial_reference={
        'wkid': 4326, 
        'latestWkid': 4326
    }
)

Add the routes back to the map using the `draw` method of the map widget once again. Make sure to scroll back up to verify the result

You can use this value for the symbol parameter `{'type': 'esriSLS', 'style': 'esriSLSSolid','color': [0,0,255,100], 'width': 6}`

In [74]:
# let make the symbol first:
route_syb = {
    'type': 'esriSLS',
    'style': 'esriSLSSolid',
    'color': [0, 0, 255, 100],
    'width': 6
}

# time to draw the result to the map
map1.draw(
    routes_fset, 
    symbol=route_syb
)

## Bonus Challenge 
for 2 Extra Points: Label the drawn Line with the distance value from the result object

In [ ]:
# optional bonus content

In [49]:
# lets use the attribute key to find the distance first
dis = result['routes']['features'][0]['attributes']['Total_Kilometers']
# converting this into a string
distance = f"{dis:.2f} KM" # aslo rounding off
print(distance)

1.12 KM


In [51]:
# now lets find the mid point in order to place this our distance 
route_path = result['routes']['features'][0]['geometry']['paths'][0]
mid_point = route_path[len(route_path)//2]  

print(mid_point)

# now we have mid point coordinates, so we will now make a feature set 
# from it as we have been doing till now

mid_point_feature = Feature(geometry={'x': mid_point[0], 'y': mid_point[1]})
mid_point_f_set = FeatureSet([mid_point_feature])

[7.645736100000022, 51.955489800000066]


In [76]:
# let make the symbol for our text
text_syb = {
    "type": "esriTS",  
    "color": [0, 0, 255, 100],
    "text": distance  # we will give the text as our input
}

# time to add this text as a label
map1.draw(
    mid_point_f_set, 
    symbol=text_syb  
)